# Custom interactive interfaces

This tutorial will demonstrate how to create custom interactive interfaces to visualize your data with `plopp`.

## A set of connected nodes

In `plopp`, think of each element in your interface as a set of interconnected nodes in a graph.
Each node can have parent nodes, children nodes, and also views attached to them (e.g. figures).

At the most basic level, a graph will contain a node (white rectangle) that provides the input data,
and a view (grey ellipse) which will be figure to display the data visually.

![graph](_static/node_graph.png)

When the data in the input node changes, the view is notified about the change.
It is then up to the view to decide whether it is interested in the notification message (most of the time it is),
and if so it requests the data from its parent node and updates the visuals on the figure.

### Nodes are callables

Nodes in the graph have to be constructed from callables.
When a view requests data from a parent node, the callable is called.
Typically, the callables will be a function that take in a data array as an input,
and returns a data array as an output.

Keeping your inputs and outputs as data arrays is useful because figure views will only accept data arrays as input.
That said, nodes that produce other outputs are very common, for example when using interactive widgets.

In the small example above, the node at the top of the graph has no parents,
and its callable is simply a `lambda` function with no arguments that just returns the input data.

## Constructing the graph

To make our graph above, we begin by generating some 2D data,
and feeding that to an `input_node`.

In [ ]:
%matplotlib widget
import plopp as pp
import scipp as sc
import numpy as np

da = pp.data.dense_data_array(ndim=2)
noise = sc.array(dims=da.dims, values=np.random.random(da.shape), unit=da.unit)
da.data = (da.data * 5.0) + noise
da

In [ ]:
a = pp.input_node(da)

Next, we give that input node as input to a two-dimensional figure:

In [ ]:
fig = pp.figure2d(a)

We can display our graph using the `show_graph` function,
which will accept as input any node in the graph.

In [ ]:
pp.show_graph(a)

As shown in the graph above, each node has a callable (in this case a `lambda` function) and an `id`.

The figure can directly be displayed in the notebook.

In [ ]:
fig

## Expanding the graph

Next, say we wish to add a gaussian smoothing step in our graph, before showing the data on the figure.
We start with the same `input_node`, but add a second node that performs the smoothing operation before attaching the figure.
Because the `gaussian_filter` function requires a kernel width `sigma` as input,
we set it using the `partial` function from the `functools` module. 

In [ ]:
a = pp.input_node(da)

from scipp.ndimage import gaussian_filter
from functools import partial
b = pp.node(partial(gaussian_filter, sigma=5))(a)

fig = pp.figure2d(b)

The resulting graph has an input node, a smoothing node, and a figure:

In [ ]:
pp.show_graph(a)

And the resulting figure displays the smoothed data:

In [ ]:
fig

## Adding interactive widgets

In the example above, the kernel size `sigma` for the gaussian smoothing was frozen to `5`.
But we would actually want to control this via a slider widget.

In this case, the smoothing node `b` now needs two inputs: the raw data, and the `sigma`.
It gets the raw data from the `input_node`, and the `sigma` from a `widget_node`,
which is coupled to a slider from the `ipywidgets` library.

In [ ]:
a = pp.input_node(da)

import ipywidgets as ipw
slider = ipw.IntSlider(min=1, max=10)
b = pp.widget_node(slider)

c = pp.node(gaussian_filter)(a, sigma=b)

fig = pp.figure2d(c)

As expected, the smoothing node `c` now has two parent nodes:

In [ ]:
pp.show_graph(a)

And we can display the figure and the slider inside the same container:

In [ ]:
ipw.VBox([slider, fig])

When something changes in one of the nodes, all the nodes below it in the graph are notified about the change (the children nodes receive a notification, and they, in turn, notify their own children).
It is then up to each view to decide whether they are interested in the notification or not (usually, most views are interested in all notifications from parents).
If they are, they request data from their parent nodes, which in turn request data from their parents, and so on, until the request has reached the top of the graph.

As a result, when the slider is dragged, the smoothing node `c` gets notified and tells the figure that a change has occured.
The figure tells node `c` that it wants updated data.
Node `c` asks nodes `a` and `b` for their data.
`a` returns the raw data, while `b` returns the integer value for the kernel size.
`c` then simply send the inputs to the `gaussian_filter` function, and forwards the result to the figure.

## Multiple views

To go one step further,
we now wish to add a second one-dimensional figure that will display the sum of the two-dimensional data along the vertical dimension.
On this figure, we would like to display both the original (unsmoothed) data, as well as the smoothed data.

In [ ]:
a = pp.input_node(da)

slider = ipw.IntSlider(min=1, max=10)
b = pp.widget_node(slider)

c = pp.node(gaussian_filter)(a, sigma=b)

fig2d = pp.figure2d(c)

# Sum the raw data along the vertical dimension
d = pp.node(sc.sum, dim='yy')(a)
# Sum the smoothed data along the vertical dimension
e = pp.node(sc.sum, dim='yy')(c)
# Give two nodes to a figure to display both on the same axes
fig1d = pp.figure1d(d, e)

We check the graph again to make sure that the one-dimensional figure has two inputs,
and that both are performing a sum along the `yy` dimension.

In [ ]:
pp.show_graph(a)

In [ ]:
ipw.VBox([slider, fig2d, fig1d])

Because the slider in only affecting the left part of the graph,
only the orange markers will update when we drag the slider.